# <span style="color:green"><center>Diplomado en Inteligencia Artificial y Aprendizaje Profundo</center></span>

# <span style="color:red"><center>Tensorborad en tensorflow</center></span>

##  Profesores

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 
3. Campo Elías Pardo Turriago, cepardot@unal.edu.co 

## Asistentes

5. Oleg Jarma, ojarmam@unal.edu.co 
6. Laura Lizarazo, ljlizarazore@unal.edu.co
7. Julieth López, julalopezcas@unal.edu.co

## Contenido

* [¿Qué es Tensorborad?]()
    * [Tablero de tensorBoard]()
* [Requerimientos]()
* [Instalación y carga]()
* [Ejemplo]()
    * [Modelo]()
    * [Graficando imaganes con TensorBoard]()
    * [Tensorboard callback]()
    * [Ejecución de Tensorboard]()
    * [Matriz de confusión en TensorBoard]()
    * [Ajuste de hiperparámetros con TensorBoard]()
* [TensorFlow Profiler]()

# ¿Qué es Tensorborad?

Tutorial tomado de [Neptune blog, Deep Dive Into TensorBoard: Tutorial With Examples](https://neptune.ai/blog/tensorboard-tutorial)

Es una herramienta que permite rastrear varias métricas como la precisión (accuracy) y los registros de los grupos de pérdida de entrenamiento o de validación. TensorBoard provee distintas aplicaciones para utilizar en experimentos de aprendizaje de maquina. Algunas de las aplicaciones que se pueden ver en las distintas pestañas del tablero de Tensorboard son:

## Tablero de tensorBoard

- **Scalars**: Muestra los cambios en la perdida y métricas cobre las epocas. Tambien puede usarse para rastrear otros valores escalares como la taza de aprendizaje y la velocidad de entrenamiento.
- **Images**: Tiene imagenes que muestran los pesos. Parandose con sobre una epoca especifica se pueden ver los pesos del modelo en esa epoca.
- **Graphs**: Muestra las capas del modelo. Se puede utilizar para revisar si la arquitectura del modelo es la que se pretende.
- **Distributions**: Muestra la distribución de los tensores. Por ejemplo, se puede ver la distribucion de los pesos y sesgos sobre cada epoca en una capa específica.
- **Histograms**: Muestra la distribución de los tensores sobre el tiempo, sobre cada epoca.
- **Proyector**: Se puede utilizar para visualizar la representación de cada vector, por ejemplo, word embeddings (la representación numérica de las palabras que captura su relación semantica) y imágenes.

# Requerimientos

In [12]:
from datetime import datetime
import itertools
import io

import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

from tensorflow import keras
from sklearn import metrics
from tensorboard.plugins.hparams import api as hp

# Instalación y carga

Se puede utlizar ´pip´ o ´conda´ para la **instalación**, observe los siguientes comandos:

`pip install tensorboard`

`conda install -c conda-forge tensorboard`

In [11]:
conda install -c conda-forge tensorboard

Note: you may need to restart the kernel to use updated packages.

Solving environment: ...working... done

## Package Plan ##


WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(143): Could not remove or rename C:\Users\LENOVO\anaconda3\pkgs\numpy-base-1.17.0-py37hc3f5095_0.tar.bz2.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(143): Could not remove or rename C:\Users\LENOVO\anaconda3\pkgs\numpy-base-1.17.0-py37hc3f5095_0\Lib\site-packages\numpy\ma\tests\test_regression.py.  Please remove this file manually (you may need to reboot to free file handles)



  environment location: C:\Users\LENOVO\anaconda3

  added / updated specs:
    - tensorboard


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    absl-py-1.0.0              |     pyhd8ed1ab_0          95 KB  conda-forge
    ca-certificates-2021.10.8  |       h5b45459_0         176 KB  conda-forge
    certifi-2021.10.8          |   py38haa244fe_1         145 KB  conda-forge
    conda-4.11.0               |   py38haa244fe_0        16.9 MB  conda-forge
    grpcio-1.42.0              |   py38hc60d5dd_0         1.9 MB
    libprotobuf-3.17.2         |       h23ce68f_1         1.9 MB
    markdown-3.3.4             |   py38haa95532_0         144 KB
    openssl-1.1.1l             |       h8ffe710_0         5.7 MB  conda-forge
    protobuf-3.17.2            |   py38hd77b12b_0         257 KB
    python_abi-3.8             |           2_cp38           4 KB  conda-forge
    tensorboard-1.15.0         

Se puede **cargar** Tensorboard utilizando Jupyter notebook, Jupyter lab o Google Colab.

In [13]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Tensorboard genera unos archivos "logs" o "registros" del codigo que se ejecuta y que deben ser guaradados.

In [14]:
log_folder = 'logs1'

En caso de querer recargar la extensión se puede utilizar el siguiente código.

In [15]:
%reload_ext tensorboard

Para limpiar los `logs` y dejar libre el folder se pueden correr los siguientes comandos:

- Para linux: `rm -rf logs`
- Para colab: `!rm -rf /logs/`
- Para windows utilizar ambos:
    - `!taskkill /f /t /im tensorboard.exe`
    - `!del /a /s /q /f logs`

Si se estan corriendo distintos experimentos, todos ellos se pueden guardar para luego compararlos creando logs guardados con una marca de tiempo utilizando:

In [16]:
# Clear out any prior log data.
!taskkill /f /t /im tensorboard.exe
!del /a /s /q /f logs

# Sets up a timestamped log directory.
log_folder = f"{log_folder}/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(log_folder)

CORRECTO: el proceso con PID 6184 (proceso secundario de PID 8852)
ha sido terminado.
CORRECTO: el proceso con PID 8852 (proceso secundario de PID 5372)
ha sido terminado.


No se pudo encontrar C:\Users\LENOVO\Downloads\logs


# Ejemplo

Usaremos TensorBoard para visualizar las metricas de un modelo. Construiremos para ello un modelo simple de clasificación de imágenes.

## Modelo

In [6]:
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0 #normalización
class_names = ['Zero','One','Two','Three','Four','Five','Six','Seven','Eight','Nine']

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

## Graficando imagenes de entrenamiento con TensorBoard

Este tablero (dashboard) tiene imágenes que muestran los pesos. Ajustando con los botones deslizantes los pesos para las distintas epocas.

Se puede usar la API de resumen de imagenes de TensorFlow para visualizar las imagenes de entrenamiento. Esto es especialmente util cuando se trabaja con datos de imágenes como en este caso. Anteriormente se habían especificado las imagenes con un tamaño de 28x28, por eso, es importante reajustar el tamaño de las imagenes antes de escribirlas en TensorFlow. Tambien se necesita especificar el canal a 1 porque las imagenes estan en escala de grises. Después, se utuliza la función **file_write** para escribir las imagenes en TensorBoard. 

En este caso las imágenes indexadas de 10 a 30 seran mostradas en TensorBoar.

In [7]:
# Guardar imagenes para mostrar en Tensorboard.
with file_writer.as_default():
    images = np.reshape(X_train[10:30], (-1, 28, 28, 1))
    tf.summary.image("20 Digits", images, max_outputs=25, step=0)

## Tensorboard callback

Para especificar la retrollamada (call back) durante el ajuste del modelo se debe importar Tensorboard. Esta retrollamada es responsable de registrar los eventos como los histogramas de activación, gráficas de resumen de las métricas o gráficos de visualización de perfilado (profiling: duración de ejecución de un código) y entrenamiento.

In [17]:
from tensorflow.keras.callbacks import TensorBoard

Creamos la retrollamada (callback) y especificamos el directorio de los registros (logs) utilizando el código `log_dir`. Otros de los parámetros que se utilizan son:

- `histogram_freq`: Es la frecuencia con la que se computa la activación y pesos de los histogramas por capas del modelo. Cuando se deja en cerp significa que los histogramas no seran computados. Para esto se debe trabajar con un conjunto de validación.
- `write_graph`:  Dicta si el gráfico sera visualizaso en TensorBoard.
- `write_images`: Cuando es verdadero (True), los pesos del modelo son visualizados como una imagen en TensorBoard
- `update_freq_`: Determina como las perdidas y metricas son escritas en TensorBoard. Cuando se establece como un entero, por ejemplo 100, las perdidas y metricas son registradas cada 100 lotes. Cuando se define por lotes, las metricas son establecidas despues de cada lote y cuando se define por epoca, se establecen despues de cada epoca.
- `profile_batch`: Determina que lotes (batches) seran medidos (profiled). Por defecto, el segundo lote es medido.
- `embeddings_freq`: Frecuencia con que las capas de embedding del modelo son visualizadas, con cero no son visualizadas.

Ajustamos el modelo llamando la retrollamada.

In [18]:
model = create_model()
optim = tf.keras.optimizers.Adam(learning_rate=0.000004)

model.compile(optimizer=optim,
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_folder, histogram_freq=1)

model.fit(x=X_train, 
        y=y_train, 
        epochs=8, 
        validation_data=(X_test, y_test), 
        callbacks=[tensorboard_callback])

Epoch 1/8
1875/1875 [==============================] - 14s 7ms/step - loss: 2.1230 - accuracy: 0.2919 - val_loss: 1.4197 - val_accuracy: 0.7575
Epoch 2/8
1875/1875 [==============================] - 12s 6ms/step - loss: 1.2958 - accuracy: 0.7487 - val_loss: 0.9102 - val_accuracy: 0.8383
Epoch 3/8
1875/1875 [==============================] - 12s 6ms/step - loss: 0.8777 - accuracy: 0.8176 - val_loss: 0.6683 - val_accuracy: 0.8685
Epoch 4/8
1875/1875 [==============================] - 12s 6ms/step - loss: 0.6725 - accuracy: 0.8509 - val_loss: 0.5430 - val_accuracy: 0.8830
Epoch 5/8
1875/1875 [==============================] - 12s 6ms/step - loss: 0.5657 - accuracy: 0.8653 - val_loss: 0.4709 - val_accuracy: 0.8932
Epoch 6/8
1875/1875 [==============================] - 12s 6ms/step - loss: 0.4985 - accuracy: 0.8771 - val_loss: 0.4255 - val_accuracy: 0.8991
Epoch 7/8
1875/1875 [==============================] - 12s 6ms/step - loss: 0.4517 - accuracy: 0.8849 - val_loss: 0.3937 - val_accuracy:

## Ejecución de Tensorboard

Se puede ejecutar con alguno de los siguientes códigos:

- Si se instaló con pip y se quiere correr en la terminal: `tensorboard --logdir=log`
- Si se va a correr en el cuaderno: `%tensorboard --logdir={log_folder}`
- Si se desea ver en el navegador: http://localhost:6006

In [ ]:
%tensorboard --logdir={log_folder}

Launching TensorBoard...

Si se desea compartir los resultados obtenidos en Tensorboard basta con oprimir el botón **upload** que pedira correr un comando similar al mostrado a continuación. Resulta mejor correrlo desde la consola o en colab.

`tensorboard dev upload --logdir 'logs2/fit/20211121-184313'`

## Matriz de confusión en TensorBoard

Usando el mismo ejemplo, puede registrar la matriz de confusión para todas las épocas. Primero, se define una función que devolverá una figura Matplotlib que mantiene la matriz de confusión.

In [11]:
def plot_to_image(figure):    
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(figure)
    buf.seek(0)

    digit = tf.image.decode_png(buf.getvalue(), channels=4)
    digit = tf.expand_dims(digit, 0)

    return digit

def plot_confusion_matrix(cm, class_names): 
    figure = plt.figure(figsize=(8, 8)) 
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Accent) 
    plt.title("Confusion matrix") 
    plt.colorbar() 
    tick_marks = np.arange(len(class_names)) 
    plt.xticks(tick_marks, class_names, rotation=45) 
    plt.yticks(tick_marks, class_names)

    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)  
    threshold = cm.max() / 2. 

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):   
        color = "white" if cm[i, j] > threshold else "black"   
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)  
    
    plt.tight_layout() 
    plt.ylabel('True label') 
    plt.xlabel('Predicted label') 

    return figure

A continuación, borramos los registros anteriores, definimos el directorio de registro para la matriz de confusión, y creamos una variable de escritor para escribir en la carpeta de registro.

In [12]:
# Clear out any prior log data.
!taskkill /f /t /im tensorboard.exe
!del /a /s /q /f logs

log_folder2 = 'logs2'
# Sets up a timestamped log directory.
log_folder2 = f"{log_folder2}/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(log_folder2)

CORRECTO: el proceso con PID 6884 (proceso secundario de PID 13552)
ha sido terminado.
CORRECTO: el proceso con PID 13552 (proceso secundario de PID 16304)
ha sido terminado.


El paso que sigue es crear una función que hará predicciones del modelo y registrara la matriz de confusión como una imagen. Después de eso, se utiliza `File_Writer` para escribir la matriz de confusión al directorio de registro.

In [13]:
def log_confusion_matrix(epoch, logs):
    predictions = model.predict(X_test)
    predictions = np.argmax(predictions, axis=1)

    cm = metrics.confusion_matrix(y_test, predictions)
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)
    
    with file_writer.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

Esto será seguido por la definición de la retrollamada de Tensorboard y el Lambdacallback. El Lambdacallback registrará la matriz de confusión en cada época. Finalmente, se corre el modelo utilizando estos dos callbacks.

In [14]:
callbacks = [
   TensorBoard(log_dir=log_folder2, 
               histogram_freq=1, 
               write_graph=True,
               write_images=True,
               update_freq='epoch',
               profile_batch=2,
               embeddings_freq=1),
   keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)
]

model.fit(X_train, y_train,
          epochs=8,
          validation_split=0.2,
          callbacks=callbacks)

Epoch 1/8
1500/1500 [==============================] - 7s 4ms/step - loss: 0.3986 - accuracy: 0.8940 - val_loss: 0.3423 - val_accuracy: 0.9121
Epoch 2/8
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3833 - accuracy: 0.8975 - val_loss: 0.3308 - val_accuracy: 0.9132
Epoch 3/8
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3693 - accuracy: 0.9008 - val_loss: 0.3209 - val_accuracy: 0.9158
Epoch 4/8
1500/1500 [==============================] - 7s 4ms/step - loss: 0.3588 - accuracy: 0.9028 - val_loss: 0.3123 - val_accuracy: 0.9173
Epoch 5/8
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3487 - accuracy: 0.9049 - val_loss: 0.3048 - val_accuracy: 0.9188
Epoch 6/8
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3408 - accuracy: 0.9082 - val_loss: 0.2980 - val_accuracy: 0.9197
Epoch 7/8
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3314 - accuracy: 0.9103 - val_loss: 0.2915 - val_accuracy: 0.9213

Now run TensorBoard and check the confusion matrix on the Images tab.

In [15]:
%tensorboard --logdir={log_folder2}

## Ajuste de hiperparámetros con TensorBoard

Tensorborad tambien se puede utilizar para visualizar la optimización de los hiperparámetros, por ejemplo, el número de lotes o la tasa de aprendizaje. Se puede revisar los hiperparámetros del modelo manualmente o usando una optimización automatizada y visualizandolos en TensorBoard. El tablero está disponible bajo la pestaña HPARAMS. Para esto se debe lipiar los registros previos e importar el paquete hparams.

In [31]:
# Clear out any prior log data.
!taskkill /f /t /im tensorboard.exe
!del /a /s /q /f logs

ERROR: no se encontr¢ el proceso "tensorboard.exe".


Archivo eliminado: C:\Users\JULIETH LOPEZ\Documents\Diplomado_IA_AP\logs\hparam_tuning\events.out.tfevents.1637626001.LAPTOP-QV7ELDEM.16304.6.v2
Archivo eliminado: C:\Users\JULIETH LOPEZ\Documents\Diplomado_IA_AP\logs\hparam_tuning\events.out.tfevents.1637627118.LAPTOP-QV7ELDEM.16304.25.v2
Archivo eliminado: C:\Users\JULIETH LOPEZ\Documents\Diplomado_IA_AP\logs\hparam_tuning\Experiment 0\events.out.tfevents.1637626005.LAPTOP-QV7ELDEM.16304.7.v2
Archivo eliminado: C:\Users\JULIETH LOPEZ\Documents\Diplomado_IA_AP\logs\hparam_tuning\Experiment 1\events.out.tfevents.1637626035.LAPTOP-QV7ELDEM.16304.8.v2
Archivo eliminado: C:\Users\JULIETH LOPEZ\Documents\Diplomado_IA_AP\logs\hparam_tuning\Experiment 10\events.out.tfevents.1637626501.LAPTOP-QV7ELDEM.16304.17.v2
Archivo eliminado: C:\Users\JULIETH LOPEZ\Documents\Diplomado_IA_AP\logs\hparam_tuning\Experiment 11\events.out.tfevents.1637626561.LAPTOP-QV7ELDEM.16304.18.v2
Archivo eliminado: C:\Users\JULIETH LOPEZ\Documents\Diplomado_IA_AP\logs\

In [32]:
logdir = "logs3"

from tensorboard.plugins.hparams import api as hp

El siguiente paso es definir los parámetros a sintonizar. En este caso, las unidades en la capa densa, la tasa de deserción (dropout rate) y la función del optimizador se sintonizarán.

In [33]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([300, 200,512]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1,0.5))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd', 'rmsprop']))

Luego, se usa `tf.summary.create_file_writer` para definir la carpeta donde los registros serán guardados.

In [34]:
METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer(f'{logdir}/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],)

Con eso, se debe definir el modelo como se hizo anteriormente sin embargo, la diferencia está en el número de neuronas en la primera capa densa, la tasa de abandono (drop out rate) y la función del optimizador, ya que estas no se codificarán.

Esto se hará en una función que se utilizará más adelante, mientras ejecute los experimentos.

In [35]:
def create_model(hparams):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS],  activation='relu'),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(10, activation='softmax')])

    model.compile(optimizer=hparams[HP_OPTIMIZER],
                  loss='sparse_categorical_crossentropy',  
                  metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=5)
    loss, accuracy = model.evaluate(X_test, y_test)
    
    return accuracy

La siguiente función que se debe crear ejecutará la función anterior utilizando los parámetros definidos anteriormente. Luego se registrará la precisión.

In [36]:
def experiment(experiment_dir, hparams):

    with tf.summary.create_file_writer(experiment_dir).as_default():
        hp.hparams(hparams)
        accuracy = create_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

Después de esto, debe ejecutar esta función para todas las combinaciones de los parámetros definidos anteriormente. Cada uno de los experimentos se almacenará en su propia carpeta.

In [37]:
experiment_no = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer,}

            experiment_name = f'Experiment {experiment_no}'
            print(f'Starting Experiment: {experiment_name}')
            print({h.name: hparams[h] for h in hparams})
            experiment(f'{logdir}/hparam_tuning/' + experiment_name, hparams)
            experiment_no += 1

Starting Experiment: Experiment 0
{'num_units': 200, 'dropout': 0.1, 'optimizer': 'adam'}
Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2514 - accuracy: 0.9271
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1104 - accuracy: 0.9668
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0773 - accuracy: 0.9762
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0590 - accuracy: 0.9812
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0627 - accuracy: 0.9789
Starting Experiment: Experiment 1
{'num_units': 200, 'dropout': 0.1, 'optimizer': 'rmsprop'}
Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2497 - accuracy: 0.9276
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1204 - accuracy: 0.9649
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0910 - accuracy: 0.9743
Epoch 4/5
1

Finalmente, se ejecuta Tensorboard para ver la visualización que se vio al comienzo de estenotebook. En la pestaña *Hparams*, se muestran todos los modelos corridos y su precisión (accuracy), dropout rate y capas densas. *Parallel Coordinates View* muestra cada corrida como una línea que se mueve a través de un eje para cada uno de los hiperparamétricos y la métrica de precisión. Al hacer clic en uno de ellos, mostrará los ensayos de los hiperparametros y *Scatter Plot View* visualiza la comparación entre los hiperparaméteres y las métricas.

In [39]:
%tensorboard --logdir={logdir}/hparam_tuning

## TensorFlow Profiler

También puede realizar un seguimiento del rendimiento de los modelos de TensorFlow utilizando la herramienta `Profiler` que resulta crucial para comprender el consumo de recursos de recursos de hardware de las operaciones de TensorFlow. Esta herramienta solo está disponible para quienes cuenten con equipos con GPU. Quienes deseen pueden hacer uso especial de la siguiente aplicación de la herramienta:

- Input pipeline analyzer: Se puede utilizar para analizar las ineficiencias en la tubería de entrada (input pipeline) del modelo.